<a href="https://colab.research.google.com/github/LasciaStare/Lab-1-EDDII/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio Estructura de datos II.

In [ ]:
!pip install graphviz

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
from collections import deque
from IPython.display import Image
from graphviz import Digraph
from typing import Any, List, Optional, Tuple

In [ ]:
# Ruta a la carpeta montada en Google Drive
folder_path = "/content/drive/MyDrive/data"

# Obtener nombres de archivos en la carpeta
types = os.listdir(folder_path)
for tipo in types:
  # Construir la ruta completa de la subcarpeta
  subcarpeta_path = os.path.join(folder_path, tipo)
  # Verificar si la subcarpeta es un directorio
  if os.path.isdir(subcarpeta_path):
    if os.path.isdir(subcarpeta_path):
        # Obtener los nombres de archivos de la subcarpeta y asignarlos directamente a una lista con el nombre de la subcarpeta
        globals()[tipo] = os.listdir(subcarpeta_path)

Nodo

In [ ]:
class Nodo:

    nodo_counter = 0
    def __init__(self, name: str,Type : str) -> None:
        Nodo.nodo_counter+=1
        self.name = name
        self.left: Optional["Nodo"] = None
        self.right: Optional["Nodo"] = None
        self.height = 1
        self.Type = Type
        self.weight = os.path.getsize(os.path.join(folder_path, Type, name))

    def __repr__(self) -> str:
        return f"{self.name}"

class AVL:

    def __init__(self) -> None:
        self.root = None
    #Step 1 Perform as a Arbol de busqueda
    def insert(self, name, Type):
        if self.root is None:
          self.root = Nodo(name, Type)
        else:
          self.root = self._insert(self.root, name, Type)

    def _insert(self, root, name, Type):
      if root is None:
          return Nodo(name, Type)
      elif name < root.name:
          root.left = self._insert(root.left, name, Type)
      else:
          root.right = self._insert(root.right, name, Type)

      # Step 2 - Update the height of the ancestor node
      root.height = 1 + max(self.getHeight(root.left),
                            self.getHeight(root.right))

      # Step 3 - Get the balance factor
      balance = self.getBalance(root)

      # Step 4 - If the node is unbalanced,
      # then try out the 4 cases
      # Case 1 - Left Left
      if balance > 1 and name < root.left.name:
          return self.rotateRight(root)

      # Case 2 - Right Right
      if balance < -1 and name > root.right.name:
          return self.rotateLeft(root)

      # Case 3 - Left Right
      if balance > 1 and name > root.left.name:
          root.left = self.rotateLeft(root.left)
          return self.rotateRight(root)

      # Case 4 - Right Left
      if balance < -1 and name < root.right.name:
          root.right = self.rotateRight(root.right)
          return self.rotateLeft(root)

      # No unbalance detected, return the current root
      return root

    def rotateLeft(self, root):

        right_child = root.right
        left_grandchild = right_child.left

        # Perform rotation
        right_child.left = root
        root.right = left_grandchild

        # Update heights
        root.height = 1 + max(self.getHeight(root.left),
                              self.getHeight(root.right))
        right_child.height = 1 + max(self.getHeight(right_child.left),
                                     self.getHeight(right_child.right))

        # Return the new root
        return right_child

    def rotateRight(self, root):

        left_child = root.left
        right_grandchild = left_child.right

        # Perform rotation
        left_child.right = root
        root.left = right_grandchild

        # Update heights
        root.height = 1 + max(self.getHeight(root.left),
                              self.getHeight(root.right))
        left_child.height = 1 + max(self.getHeight(left_child.left),
                                    self.getHeight(left_child.right))

        # Return the new root
        return left_child

    def search(self, elem: Any) -> Tuple[Optional["Nodo"], Optional["Nodo"]]:
        p, pad = self.root, None
        while p is not None:
            if elem == p.name:
                return p
            elif elem < p.name:
                pad = p
                p = p.left
            else:
                pad = p
                p = p.right
        return p

    def delete(self, name):
        self.root = self.__delete(self.root, name)

    def __delete(self, root, name):
        if not root:
            return root

        if name < root.name:
            root.left = self.__delete(root.left, name)
        elif name > root.name:
            root.right = self.__delete(root.right, name)
        else:
            if root.left is None:
                temp = root.right
                root = None
                return temp
            elif root.right is None:
                temp = root.left
                root = None
                return temp
            temp = self.__min_value_node(root.right)
            root.name = temp.name
            root.right = self.__delete(root.right, temp.name)

        if root is None:
            return root

        root.height = 1 + max(self.getHeight(root.left), self.getHeight(root.right))
        balance = self.getBalance(root)

        # Casos de rotación
        if balance > 1 and self.getBalance(root.left) >= 0:
            return self.rotateRight(root)

        if balance < -1 and self.getBalance(root.right) <= 0:
            return self.rotateLeft(root)

        if balance > 1 and self.getBalance(root.left) < 0:
            root.left = self.__left_rotate(root.left)
            return self.rotateRight(root)

        if balance < -1 and self.getBalance(root.right) > 0:
            root.right = self.rotateRight(root.right)
            return self.rotateLeft(root)

        return root

    def __min_value_node(self, node):
        current = node
        while current.left:
            current = current.left
        return current

    def searchByCriteria(self, node: "Nodo", Type, min, max, list1 = None) -> None:
        if list1 is None:
          list1 = []

        if node is not None:
            if node.Type == Type and (min < node.weight < max):
              list1.append(node)
            self.searchByCriteria(node.left, Type, min, max, list1)
            self.searchByCriteria(node.right, Type, min, max, list1)
        return list1

    # Imprimir recorrido por niveles
    def printLevelOrder(self):
        h = self.root.height
        for i in range(1, h+1):
            print(f"Nivel {i}")
            self.printCurrentLevel(self.root, i)

    def printCurrentLevel(self, root, level):
        if root is None:
            return
        if level == 1:
            print(root.name, end=" | ")
        elif level > 1:
            self.printCurrentLevel(root.left, level-1)
            self.printCurrentLevel(root.right, level-1)

    def getHeight(self, root):
        if not root:
            return 0

        return root.height

    def getBalance(self, root):
        if not root:
            return 0

        return self.getHeight(root.left) - self.getHeight(root.right)

    def obtener_nivel(self, name: str) -> int:
        return self._obtener_nivel_util(self.root, name, 0)

    def _obtener_nivel_util(self, nodo: Nodo, name: str, current_level: int) -> int:
        if not nodo:
            return -1  # Si el nodo no se encuentra, devuelve -1

        if name == nodo.name:
            return current_level

        elif name < nodo.name:
            return self._obtener_nivel_util(nodo.left, name, current_level + 1)

        else:
            return self._obtener_nivel_util(nodo.right, name, current_level + 1)

        # Hallar el padre
    def findParent(self, root, nodo): #nodo = resultado de la busqueda
        if not root:
            return None

        # Si el nodo es la raíz, no tiene padre
        if root == nodo:
            return None

        # Verificar si el nodo buscado está a la izquierda de la raíz
        if root.left and root.left == nodo:
            return root

        # Verificar si el nodo buscado está a la derecha de la raíz
        if root.right and root.right == nodo:
            return root

        # Recursivamente buscar en el subárbol izquierdo
        if root.left:
            parent = self.findParent(root.left, nodo)
            if parent:
                return parent

        # Recursivamente buscar en el subárbol derecho
        if root.right:
            parent = self.findParent(root.right, nodo)
            if parent:
                return parent

        # Si no se encuentra en ningún subárbol, retornar None
        return None

    #Hallar el abuelo
    def findGrandParent(self, root, nodo):
        if not root:
            return None

        parent = self.findParent(root, nodo)
        if parent:
            grandparent = self.findParent(root, parent)
            return grandparent
        else:
            return None
    #Hallar tio
    def findUncle(self, root, nodo):
        if not root:
            return None

        parent = self.findParent(root, nodo)
        if parent:
            grandparent = self.findParent(root, parent)
            if grandparent:
                if grandparent.left == parent:
                    return grandparent.right
                else:
                    return grandparent.left
        return None


In [ ]:
def visualize_tree(root):
    dot = Digraph()

    def add_nodes_edges(node):
        if node:
            dot.node(str(node.name), label=str(node.name))
            if node.left:
                dot.node(str(node.left.name), label=str(node.left.name))
                dot.edge(str(node.name), str(node.left.name))
                add_nodes_edges(node.left)
            if node.right:
                dot.node(str(node.right.name), label=str(node.right.name))
                dot.edge(str(node.name), str(node.right.name))
                add_nodes_edges(node.right)

    add_nodes_edges(root)
    dot.render('binary_tree', format='png', cleanup=True)
    dot.view()


Submenu

In [ ]:
def add_opt(name):
    print("Operaciones adicionales para el nodo seleccionado:")
    print("a. Obtener el nivel del nodo")
    print("b. Obtener el factor de balanceo (equilibrio) del nodo")
    print("c. Encontrar el padre del nodo")
    print("d. Encontrar el abuelo del nodo")
    print("e. Encontrar el tío del nodo")

    sub_choice = input("Seleccione una operación adicional: ")
    if sub_choice == "a":
        print("Nivel del nodo:", Tree.obtener_nivel(name))
    elif sub_choice == "b":
        print("Factor de balanceo del nodo:", Tree.getBalance(Tree.search(name)))
    elif sub_choice == "c":
        print("Padre del nodo:", Tree.findParent(Tree.root,Tree.search(name)))
    elif sub_choice == "d":
        print("Abuelo del nodo:", Tree.findGrandParent(Tree.root,Tree.search(name)))
    elif sub_choice == "e":
        print("Tío del nodo:", Tree.findUncle(Tree.root,Tree.search(name)))
    else:
        print("Opción no válida. Por favor, seleccione una opción válida.")


Menú

In [ ]:
 # Función para mostrar el menú
def show_menu():
    print("Menú:")
    print("1. Insertar un nodo")
    print("2. Eliminar un nodo")
    print("3. Buscar un nodo")
    print("4. Buscar nodos por criterios")
    print("5. Mostrar recorrido por niveles")
    print("6. Salir")


Tree = AVL()
root = None

while True:
    show_menu()
    choice = int(input("Seleccione una opción: "))

    if choice == 1:
      try:
        same_type = input("¿Todos los archivos serán del mismo tipo? (s/n): ").lower() == 's'
        Type = None
        if same_type:
            Type = input("Ingrese el tipo de archivo: ")
        n = int(input("Cuantos archivos va a insertar: "))  # Solicitar el número de archivos a insertar
        for _ in range(n):
            name = input("Ingrese el nombre del archivo: ")
            if not same_type:
                file_type = input("Ingrese el tipo del archivo: ")
            Tree.insert(name, Type)
            visualize_tree(Tree.root)
        print("Nodos insertados con éxito.")
      except FileNotFoundError:
        print("Al menos uno de los archivos no se encontró.")


    elif choice == 2:
        name = input("Ingrese el nombre del nodo a eliminar: ")
        Tree.delete(name)
        visualize_tree(Tree.root)

    elif choice == 3:
        name = input("Ingrese el nombre del nodo a buscar: ")
        add_opt(name)

    elif choice == 4:

      Type = input("Ingrese el tipo del nodo a buscar: ")
      min = int(input("Ingrese el mínimo tamaño del nodo a buscar: "))
      max = int(input("Ingrese el máximo tamaño del nodo a buscar: "))

      print("La lista resultante es la siguiente: ")
      resultados = Tree.searchByCriteria(Tree.root, Type, min, max)

      if resultados:  # Verificar si la lista no está vacía
        print(resultados)
        name = input("Seleccione uno de los nodos (escribiendo el nombre):")
        add_opt(name)
      else:
        print("No hay nodos con esos criterios.")

    elif choice == 5:
        Tree.printLevelOrder()

    elif choice == 6:
        print("Saliendo del programa...")
        break

    else:
        print("Opción no válida. Por favor, seleccione una opción válida.")


Menú:
1. Insertar un nodo
2. Eliminar un nodo
3. Buscar un nodo
4. Buscar nodos por criterios
5. Mostrar recorrido por niveles
6. Salir
Seleccione una opción: 1
¿Todos los archivos serán del mismo tipo? (s/n): s
Ingrese el tipo de archivo: cars
Cuantos archivos va a insertar: 4
Ingrese el nombre del archivo: carsgraz_001.bmp
Ingrese el nombre del archivo: carsgraz_004.bmp
Ingrese el nombre del archivo: carsgraz_006.bmp
Ingrese el nombre del archivo: carsgraz_002.bmp
Nodos insertados con éxito.
Menú:
1. Insertar un nodo
2. Eliminar un nodo
3. Buscar un nodo
4. Buscar nodos por criterios
5. Mostrar recorrido por niveles
6. Salir
Seleccione una opción: 1
¿Todos los archivos serán del mismo tipo? (s/n): s
Ingrese el tipo de archivo: cats
Cuantos archivos va a insertar: 5
Ingrese el nombre del archivo: cat.1.jpg
Ingrese el nombre del archivo: cat.10.jpg
Ingrese el nombre del archivo: cat.3.jpg
Ingrese el nombre del archivo: cat.5.jpg
Ingrese el nombre del archivo: cat.4.jpg
Nodos insertados